<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/EmployeePrimaryDepartment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count

#write a code to identify the primary dept of a employee , if the employee has more than one dept than then Primary Flag would he his dept and if there is only one dept and flag is N then it would be primary dep
# Create Spark session
spark = SparkSession.builder.appName("EmployeeFlag").getOrCreate()

# Define the schema and data
employee_data = [
    (1, 1, 'N'),
    (2, 1, 'Y'),
    (2, 2, 'N'),
    (3, 3, 'N'),
    (4, 2, 'N'),
    (4, 3, 'Y'),
    (4, 4, 'N')
]

employee_columns = ["Employee_ID", "Department_ID", "Primary_Flag"]

# Create DataFrame
employee_df = spark.createDataFrame(employee_data, schema=employee_columns)

# Step 1: Calculate the count of records per Employee_ID
c_df = employee_df.groupBy("Employee_ID").agg(count("*").alias("CNT"))

# Step 2: Join with original DataFrame and compute the flag
fla_df = employee_df.join(c_df, "Employee_ID").withColumn(
    "FLAG",
    when((col("CNT") > 1) & (col("Primary_Flag") == 'Y'), 'YY')
    .when((col("CNT") == 1) & (col("Primary_Flag") == 'N'), 'YN')
    .otherwise('N')
)

# Step 3: Filter for rows with FLAG 'YY' or 'YN'
result_df = fla_df.filter((col("FLAG") == 'YY') | (col("FLAG") == 'YN'))

# Show the result
result_df.show()